# Telia Workshop: Day 2

## Agenda

### 1.ODM intro -> slides
### 2. Loading results of ODM processing step 1-3
### 3. Exporting ODM results
### 4. Visualizing ODM in qgis
### 5. Producing via-filters
### 6. Additional analyses


In [1]:
import sys

import csv
import json
import time
import datetime
from geopy.distance import great_circle
from ast import literal_eval
import pandas as pd
import numpy as np
import mzgeohash
import geohash
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import glob
import pyspark.sql.functions as func
from pyspark.sql.functions import udf


In [2]:
#define path
datadir="/telia_se/"
resultsdir="./../results/"
qgisdir="./../qgis/"
tabledir="./../additional_data/"
custom_result_dir="hdfs://master-1.telia.motionlogic.de:8020/telia_se/2017/custom_aggregations/odm_result/"

## Loading results of ODM processing step 1-3
### 2.1. Aggregator output

In [3]:
#load aggregator output
df=sqlContext.read.parquet("/telia_se/2016/11/15/aggregated_events")

print "loading aggregator outout for /2016/11/15/"
print "number of subscribers: " + str(df.count())
df.show()

loading aggregator outout for /2016/11/15/
number of subscribers: 2588333
+--------------------+--------------------+----------------+----------------+--------------------+----+
|         area_id_arr|         cell_id_arr|      crm_fields|    other_fields|       timestamp_arr|IMSI|
+--------------------+--------------------+----------------+----------------+--------------------+----+
|[17, 17, 17, 17, ...|[25917727, 257623...|Map(dummy -> -1)|Map(dummy -> -1)|[1479177720, 1479...|   0|
|[57, 57, 57, 57, ...|[563, 563, 563, 5...|Map(dummy -> -1)|Map(dummy -> -1)|[1479168480, 1479...|   1|
|[235, 235, 235, 2...|[15481, 15481, 15...|Map(dummy -> -1)|Map(dummy -> -1)|[1479174600, 1479...|   2|
|[833, 833, 833, 8...|[106443, 106443, ...|Map(dummy -> -1)|Map(dummy -> -1)|[1479168360, 1479...|   3|
|[25, 25, 25, 25, ...|[27272223, 272722...|Map(dummy -> -1)|Map(dummy -> -1)|[1479168120, 1479...|   4|
|[4630, 4630, 152,...|[2644, 2644, 1043...|Map(dummy -> -1)|Map(dummy -> -1)|[1479173460, 1479

## Loading results of ODM processing step 1-3
### 2.2. odm raw

In [5]:
#load odm raw
df=sqlContext.read.parquet("/telia_se/2016/11/15/odm_raw_return")

print "loading raw trips for /2016/11/15/"
print "number of trips: " + str(df.count())

df.show()

loading raw trips for /2016/11/15/
number of trips: 2591816
+--------------------+----+----------+---------+----------+---------+----------------+----------------+
|               steps|IMSI|home_cilac| distance|  duration|    speed|      crm_fields|    other_fields|
+--------------------+----+----------+---------+----------+---------+----------------+----------------+
|[Map(timestamp ->...|   0|34547-4650|34.372818| 2.0622222|16.667852|Map(dummy -> -1)|Map(dummy -> -1)|
|[Map(timestamp ->...|   0|34547-4650| 5.378944|0.35555556| 15.12828|Map(dummy -> -1)|Map(dummy -> -1)|
|[Map(timestamp ->...|   1|    643-57|11.786726|0.53555554| 22.00841|Map(dummy -> -1)|Map(dummy -> -1)|
|[Map(timestamp ->...|   6|  444-4640|31.396418|      0.32| 98.11381|Map(dummy -> -1)|Map(dummy -> -1)|
|[Map(timestamp ->...|   6|  444-4640| 38.91192| 0.5258333| 74.00047|Map(dummy -> -1)|Map(dummy -> -1)|
|[Map(timestamp ->...|   7| 5513-4516|3.5990005| 0.5688889| 6.326368|Map(dummy -> -1)|Map(dummy -> -1)|
|[Ma

In [6]:
df_trips=df.limit(10).select("steps").rdd.flatMap(lambda x: x).collect()
print "one event: "
print df_trips[0][0]
print "******"
print "one trip:, length:  " + str(len(df_trips[0])) + " events"
print df_trips[0]



one event: 
{u'had_coord': u'true', u'timestamp': u'1479227392', u'cell_id': u'632-235'}
******
one trip:, length:  17 events
[{u'had_coord': u'true', u'timestamp': u'1479227392', u'cell_id': u'632-235'}, {u'had_coord': u'false', u'timestamp': u'1479227437', u'cell_id': u'10204-131'}, {u'had_coord': u'false', u'timestamp': u'1479227437', u'cell_id': u'10204-131'}, {u'had_coord': u'false', u'timestamp': u'1479227438', u'cell_id': u'10204-131'}, {u'had_coord': u'true', u'timestamp': u'1479229312', u'cell_id': u'6580-4640'}, {u'had_coord': u'true', u'timestamp': u'1479229312', u'cell_id': u'6580-4640'}, {u'had_coord': u'true', u'timestamp': u'1479229312', u'cell_id': u'6580-4640'}, {u'had_coord': u'false', u'timestamp': u'1479229312', u'cell_id': u'26124044-235'}, {u'had_coord': u'true', u'timestamp': u'1479229312', u'cell_id': u'6580-4640'}, {u'had_coord': u'true', u'timestamp': u'1479229347', u'cell_id': u'6580-4640'}, {u'had_coord': u'true', u'timestamp': u'1479229350', u'cell_id': u'6

## Loading results of ODM processing step 1-3
### 2.3. odm agg

In [7]:
#load odm agg
df=sqlContext.read.parquet("/telia_se/2016/11/15/odm_agg_filtered/cil_od")

print "loading raw trips for /2016/11/15/"
print "number of entries: " + str(df.count())

df.show()

loading raw trips for /2016/11/15/
number of entries: 2518171
+-------------+--------------------+-----------+
|cilac_in_list|        od_hours_key|cell_counts|
+-------------+--------------------+-----------+
|        false|[1652-72,4731-62,...|          1|
|        false|[38898-4650,24463...|          1|
|        false|[33000-4670,12462...|          1|
|        false|[11953-265,47750-...|          1|
|         true|[1091-62,6322-232...|          1|
|        false|[17611-47,361-47,...|          1|
|        false|[2256-4630,2261-4...|          1|
|        false|[11672-255,12411-...|          1|
|        false|[24742-172,3317-4...|          1|
|        false|[6269-4640,3633-2...|          1|
|        false|[17673-205,4431-1...|          1|
|        false|[3482-252,2782-11...|          1|
|        false|[21163-245,35245-...|          1|
|        false|[5562-222,211-102...|          1|
|        false|[1401-122,24583-6...|          1|
|        false|[56440-2527,3873-...|          1|
|      

## Loading results of ODM processing step 1-3
### 2.4. odm results

In [ ]:
#load odm results output
df=sqlContext.read.parquet("/telia_se/2017/09/06/odm_result/Telia_SE_MLTC_single_sum")
print "loading output of step 3 (odm_result) for /2017/09/06/"

print "number of entries: " + str(df.count())

df.show()

## 3. Export ODM results


In [4]:
#save results of custom ODM aggregation


fn="custom_odm_result_application_1484842313350_0655"
df=sqlContext.read.parquet(custom_result_dir+fn)
ch=df.toPandas()
ch.to_csv(resultsdir+fn+".csv",index=False)


In [5]:
print ch.head(3)

  cilac_in_list  count destination origin time_destination time_origin
0         False      8        8564   6186            00-24       00-24
1          True     29        8073   7930            00-24       00-24
2         False     93        5091   4852            00-24       00-24

[3 rows x 6 columns]


## 4. Visualization in QGIS

In [10]:
#load BSE centroids
cc=pd.read_csv(tabledir+"BSE_south_centroids.csv")
#load centroids of Traffic cells
vz=pd.read_csv(tabledir+"Traffic_cells_south_Centroids.csv")



In [35]:
print save ODM results as qgis-readable textfile
#make qgis
fn="custom_odm_result_application_1484842313350_0655"

dff=pd.read_csv(resultsdir+fn+".csv")
dff=pd.merge(dff,vz,left_on="origin",right_on="TrafficCel")
dff=pd.merge(dff,vz,left_on="destination",right_on="TrafficCel")
if "cilac_in_list" in dff.columns:
    with open (qgisdir+fn+'.txt','w') as f:
        f.write("origin | destination | Geom | departure| arrival | cilac_in_list | count\n ")
        for c in range (0,len(dff)):                        
            f.write(str(dff["origin"][c]) + '|' + str(dff["destination"][c]) + '|' +\
                    "LINESTRING (" + str(dff["X_x"][c])+ ' ' + str(dff["Y_x"][c])+','\
                    + str(dff["X_y"][c])+ ' ' + str(dff["Y_y"][c])+ ")"\
                    + '|'+ str(dff["time_origin"][c])\
                    + '|'+ str(dff["time_destination"][c])\
                    + '|'+ str(dff["cilac_in_list"][c])\
                    + '|'+ str(dff["count"][c]) +'\n')

else:  

    with open (qgisdir+fn+'.txt','w') as f:
        f.write("origin | destination | Geom | departure| arrival | count\n ")
        for c in range (0,len(dff)):                        
            f.write(str(dff["origin"][c]) + '|' + str(dff["destination"][c]) + '|' +\
                    "LINESTRING (" + str(dff["X_x"][c])+ ' ' + str(dff["Y_x"][c])+','\
                    + str(dff["X_y"][c])+ ' ' + str(dff["Y_y"][c])+ ")"\
                    + '|'+ str(dff["time_origin"][c])\
                    + '|'+ str(dff["time_destination"][c])\
                    + '|'+ str(dff["count"][c]) +'\n')


## 4. Producing via-filters

In [13]:
#some functions
def make_geohash(p):
    return geohash.encode(p[0],p[1])
def get_geohash_neighbours(gh,precision):    
    return mzgeohash.neighbors(gh[0:precision]).values()
    
def compare_geohash(gh_list,precision,long_gh):
    res=[i for i in gh_list if i[:precision]==long_gh[:precision]]
    if len(res)>0:
        return True
    else:
        return False
def cell_finder(p,precision=5,maxdist=3):
    gh=make_geohash(p)
    gh_list=get_geohash_neighbours(gh,precision)
    temp=cc.copy()
    temp["in"]=temp.apply(lambda row:compare_geohash(gh_list,precision,row.geohash),axis=1)
    temp=temp[temp["in"]==True]
    temp["dist"]=temp.apply(lambda row:great_circle((p[0],p[1]),(row.X,row.Y)).km,axis=1)
    temp=temp[temp["dist"]<=maxdist]
    return list(temp["CI-LAC"].values)


In [16]:
#ODM via preparation: find target cell according to radius around location
#select cells in radius of maxd km
cell_list_name="via_location_1.csv"

#Gullmarsplan
t1=[18.0792,59.2986]

maxd=2
via_cells1=cell_finder(t1,5,maxd)
print len(via_cells1)
print str(len(via_cells1))+ " mobile cells found in distance " + str(maxd) + " km  from location (lat " +  str(t1[0]) + ", long "+ str(t1[1])+")"
fc=pd.DataFrame(via_cells1)
fc.to_csv(resultsdir+cell_list_name,index=False)

print "upload file to appliance and specify path in config"

893
893 mobile cells found in distance 2 km  from location (lat 18.0792, long 59.2986)
upload file to appliance and specify path in config
